In [66]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload


from graphs import *
import altair as alt
data = get_accident_data(fname="dataset_v1.csv",sample=False)
accident_data = get_weather_data(data,fname = "weather2018.csv")

data.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\PcCom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'ON STREET NAME',
       'CROSS STREET NAME', 'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'date', 'HOUR', 'weekday', 'weekday_name', 'weekend', 'week_x', 'month',
       'week_y', 'week', 'dayname', 'monthname', 'num_days_in_month',
       'fulldate', 'INJURED', 'index', 'BoroCode', 'name', 'Shape_Leng',
      

,index_left,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,monthname,num_days_in_month,fulldate,INJURED,index,BoroCode,name,Shape_Leng,Shape_Area,month-week
0,423,2018-09-13T00:00:00Z,2023-12-28 17:30:00,STATEN ISLAND,10306.0,40.576324,-74.112830,"(40.576324, -74.11283)",SOUTH RAILROAD AVENUE,OTIS AVENUE,...,September,30,"September 13, Thursday",without injuries,0,5,Staten Island,330470.010332,1.623820e+09,9-3
0,668,2018-09-18T00:00:00Z,2023-12-28 10:28:00,NaN,NaN,40.599810,-74.178246,"(40.59981, -74.178246)",TRAVIS AVENUE,VICTORY BOULEVARD,...,September,30,"September 18, Tuesday",without injuries,0,5,Staten Island,330470.010332,1.623820e+09,9-4
0,1965,2018-08-01T00:00:00Z,2023-12-28 15:30:00,STATEN ISLAND,10309.0,40.530620,-74.194550,"(40.53062, -74.19455)",CHISHOLM STREET,AMBOY ROAD,...,August,31,"August 1, Wednesday",without injuries,0,5,Staten Island,330470.010332,1.623820e+09,8-1
0,2287,2018-07-29T00:00:00Z,2023-12-28 12:25:00,NaN,NaN,40.625175,-74.149440,"(40.625175, -74.14944)",NaN,NaN,...,July,31,"July 29, Sunday",without injuries,0,5,Staten Island,330470.010332,1.623820e+09,7-5
0,2951,2018-06-26T00:00:00Z,2023-12-28 08:20:00,NaN,NaN,40.610428,-74.110970,"(40.610428, -74.11097)",STATEN ISLAND EXPRESSWAY,NaN,...,June,30,"June 26, Tuesday",without injuries,0,5,Staten Island,330470.010332,1.623820e+09,6-5


In [67]:
colors = {"bg": "#eff0f3", "col1": "#d8b365", "col2": "#5ab4ac"}
w = 600
h = 400
ratio = 0.2
# accident_data = get_weather_data(data,fname = "weather.csv")
#print(accident_data.columns)
selection_cond = alt.selection_point(on="click",  fields=["conditions"])
selection_acc_map = alt.selection_interval()
selection_buro = alt.selection_point(fields=["name"])
selection_vehicle = alt.selection_point(on="click", fields=["VEHICLE TYPE CODE 1"])
time_brush = alt.selection_point(fields=["HOUR"])
#day_brush = alt.selection_interval(encodings=['x'])

# weekday_dropdown = alt.binding_select(options=[[0,1,2,3,4],[5,6]],name='weekday',labels=['weekday','weekend'])
#selection_weekday = alt.selection_point(fields=['weekday'])
selection_weekday = alt.selection_point(fields=["dayname"])
selection_month = alt.selection_point(fields=["monthname"])
selection_acc_factor = alt.selection_point(fields=['CONTRIBUTING FACTOR VEHICLE 1'])
selection_injured = alt.selection_multi(fields=["INJURED"])


# make month dropdownget
#month_dropdown = alt.binding_select(options=[[6,7,8,9],6,7,8,9],name='month', labels=['All','June','July','August','September'])

# selection_day = alt.selection_point(fields=['weekday','month-week'], empty=True)
# selection_day_aux = alt.selection_point(fields=['weekday','month','week'],empty=False)

cols = [
    'CRASH DATE',
    'LATITUDE',
    'LONGITUDE',
    'date',
    'fulldate',
    'HOUR',
    'week',
    'weekday',
    'month',
    'name',
    'conditions',
    'VEHICLE TYPE CODE 1',
    'CONTRIBUTING FACTOR VEHICLE 1',
    'dayname',
    'monthname',
    'num_days_in_month'
]
# select only cols
accident_data = accident_data[cols]

In [65]:
w=1000
geo_view=get_map_chart(accident_data, selection_buro, selection_acc_map, selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor, w=w*.7,ratio=.7)
weather = get_weather_chart(accident_data,selection_buro,selection_acc_map,selection_cond,selection_month,selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,  h=200,w=w*0.8,ratio=0.8)
calendar = get_calendar_chart(accident_data,selection_buro,selection_acc_map,selection_cond,selection_month,selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,  h=399,w=w*0.3)
vehicles = get_vehicle_chart(accident_data, selection_buro,selection_acc_map,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,h=200+4,w=w*.3)
time_of_day = get_time_of_day_chart(accident_data, selection_buro,selection_acc_map,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,h=399)
acc_factor = get_factor_chart(accident_data, selection_buro,selection_acc_map,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,h=399,w=w*0.3)
months = get_month_chart(accident_data, selection_buro,selection_acc_map,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush, selection_injured, selection_acc_factor,h=399,w=w*0.3)

((geo_view[0] | (geo_view[1] & vehicles)) & (weather | acc_factor).resolve_scale(color="independent")  & (months | calendar| (time_of_day).resolve_scale(color='independent')))

alt.VConcatChart(...)

####################################

In [18]:
accident_data["fulldate"]

0       September 13, Thursday
1       September 13, Thursday
2       September 13, Thursday
3       September 13, Thursday
4       September 13, Thursday
                 ...          
3757         June 23, Saturday
3758         June 23, Saturday
3759         June 23, Saturday
3760         June 23, Saturday
3761         June 23, Saturday
Name: fulldate, Length: 3762, dtype: object

In [10]:
accident_data["fulldate"] = accident_data["monthname"] + " " + accident_data["date"].dt.day.astype(str) + ", " + accident_data["dayname"]


0       September 13, Thursday
1       September 13, Thursday
2       September 13, Thursday
3       September 13, Thursday
4       September 13, Thursday
                 ...          
3757         June 23, Saturday
3758         June 23, Saturday
3759         June 23, Saturday
3760         June 23, Saturday
3761         June 23, Saturday
Name: fulldate, Length: 3762, dtype: object